<a href="https://colab.research.google.com/github/kzkzgit/video_to_bvh/blob/master/video_to_bvh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Initial imports, install, initializations 
####Be patient it'll take some time :)

In [0]:
!git clone https://github.com/Dene33/keras_Realtime_Multi-Person_Pose_Estimation.git
!pip install configobj
!apt install ffmpeg
!apt-get install blender

!apt install python2.7 python-pip
!git clone https://github.com/Dene33/hmr.git
!wget https://people.eecs.berkeley.edu/~kanazawa/cachedir/hmr/models.tar.gz && tar -xf models.tar.gz
!mv models hmr/
!pip2 install -r hmr/requirements.txt
!mkdir hmr/output
!mkdir hmr/output/csv
!mkdir hmr/output/images
!mkdir hmr/output/csv_joined
!mkdir hmr/output/bvh_animation

import os
os.chdir('keras_Realtime_Multi-Person_Pose_Estimation')
!bash model/get_keras_model.sh
!mkdir sample_jsons
!mkdir sample_videos
!mkdir sample_images

exec(open('model_load.py').read())
!pip2 install opendr==0.77

##Upload video

In [0]:
#upload video
exec(open('upload_videos.py').read())

Saving borelo500ms.mp4 to borelo500ms.mp4
User uploaded file "borelo500ms.mp4" with length 157323 bytes


##Process the video
####<font color='green'>`24`</font> in `video_to_images.sh` stands for desired fps rate at which you want to convert video to images

In [0]:
# 1. removes the duplicated 'log_dir' registration
!sed -i '59d' ../hmr/src/config.py

# 2. add None as the first auguments of tf.name_scope for these three files
!sed -i '40d' ../hmr/src/models.py
!sed -i '40i \    with tf.name_scope(None, "Encoder_resnet", [x]):' ../hmr/src/models.py
!sed -i '21d' ../hmr/src/tf_smpl/batch_lbs.py
!sed -i '21i \    with tf.name_scope(None, "batch_skew", [vec]):' ../hmr/src/tf_smpl/batch_lbs.py
!sed -i '107d' ../hmr/src/tf_smpl/batch_lbs.py
!sed -i '107i \    with tf.name_scope(None, "batch_forward_kinematics", [Rs, Js]):' ../hmr/src/tf_smpl/batch_lbs.py

# 3. cast the sorting key to string
!sed -i '194d' ../hmr/demo.py
!sed -i '194i \  all_files.sort(key=lambda x: str(x.split("/")[-1].split("/")[0]))' ../hmr/demo.py

In [0]:
os.chdir('keras_Realtime_Multi-Person_Pose_Estimation')
!pwd

In [0]:
#convert to images, specify fps rate
#!bash video_to_images.sh 1

#2d pose estimation. For each image creates corresponding .json file with format
#similar to output .json format of openpose (https://github.com/CMU-Perceptual-Computing-Lab/openpose)
#exec(open('2d_pose_estimation.py').read())

#3d pose estimation
os.chdir('..')
!bash hmr/3dpose_estimate.sh

#convert estimated .csv files to bvh
!blender --background hmr/csv_to_bvh.blend -noaudio -P hmr/csv_to_bvh.py

##Download .bvh

In [0]:
from google.colab import files
files.download('hmr/output/bvh_animation/estimated_animation.bvh')

##Display results of 3d pose estimation (optional)

In [0]:
from IPython.display import Image, display

images = os.listdir('hmr/output/images')
extensions = {".jpg", ".png", ".gif", ".bmp", ".jpeg"}

for image in sorted(images):
  for ext in extensions:
    if image.endswith(ext):
      display(Image('hmr/output/images/'+image, width=512))

##Clear all the generated data if you want to process new video

In [0]:
!rm keras_Realtime_Multi-Person_Pose_Estimation/sample_images/*
!rm keras_Realtime_Multi-Person_Pose_Estimation/sample_jsons/*
!rm keras_Realtime_Multi-Person_Pose_Estimation/sample_videos/*
!rm hmr/output/bvh_animation/*
!rm hmr/output/csv/*
!rm hmr/output/csv_joined/*
!rm hmr/output/images/*

####Based on:


1.   **End-to-end Recovery of Human Shape and Pose** by *Angjoo Kanazawa, Michael J. Black, David W. Jacobs, Jitendra Malik *
https://github.com/akanazawa/hmr  

  Corresponding fork by *Denis Cera*: https://github.com/Dene33/hmr

2.   **Realtime Multi-Person 2D Pose Estimation using Part Affinity Fields** by *Zhe Cao, Tomas Simon, Shih-En Wei, Yaser Sheikh* \
Keras implementation by *Michal Faber*: https://github.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation

  Corresponding fork by *Denis Cera*: https://github.com/Dene33/keras_Realtime_Multi-Person_Pose_Estimation


